In [ ]:
%%capture 
#Remove os outputs de instalação das bibliotecas
!pip install "google-cloud-aiplatform>=1.38"
!pip install google-cloud-storage PyPDF2
import vertexai
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import storage
from google.cloud import speech
from vertexai.preview.generative_models import GenerativeModel, Part
from google.cloud import storage
import io
from PyPDF2 import PdfReader

In [ ]:
def extrair_texto_gcs_para_pdf(nome_bucket, nome_arquivo):
    cliente = storage.Client()
    bucket = cliente.get_bucket(nome_bucket)
    blob = bucket.blob(nome_arquivo)
    byte_stream = io.BytesIO()
    blob.download_to_file(byte_stream)
    
    # Abaixo fica a leitura dos bytes que estão no pdf identificado
    byte_stream.seek(0)
    leitor_de_pdf = PdfReader(byte_stream)
    texto = ''
    for num_pag in range(len(leitor_de_pdf.pages)):
        texto += leitor_de_pdf.pages[num_pag].extract_text()
    
    return texto

# A leitura será a partir de um arquivo que está no Google Cloud Storage
gcs_nome_bucket = os.getenv('BUCKET_NAME')
gcs_nome_arquivo = os.getenv('FILE_NAME')
texto_curriculo_completo = extrair_texto_gcs_para_pdf(gcs_nome_bucket, gcs_nome_arquivo)
print(texto_curriculo_completo)

In [ ]:
def tradutor_resumidor_curriculos(nome_projeto: str, localizacao_projeto: str, texto_curriculo_completo: str) -> str:
    vertexai.init(project=nome_projeto, location=localizacao_projeto)
    
    multimodal_model = GenerativeModel("gemini-pro-vision")
    # Seu prompt pode também estar em outras línguas que a tarefa será executada também, como em inglês
    response = multimodal_model.generate_content(
        [
            """Você fará o papel de um recrutador do Linkedin, resuma os currículos apresentados, traduza eles para português brasileiro
            e separe as informações em tópicos, seu resumo deve conter os seguintes campos: Nome do candidato, localização, introdução,
            habilidades, educação, experiências prévias, certificados e experiências voluntárias, nesta ordem específica,
            abaixo está o currículo:""",
            texto_curriculo_completo
        ]
    )
    print(response.candidates[0].content.parts[0].text)
    return response.candidates[0].content.parts[0].text

nome_projeto = os.getenv('PROJECT_NAME')
localizacao_projeto = os.getenv('PROJECT_LOCATION')
curriculo_resumido = tradutor_resumidor_curriculos(nome_projeto, localizacao_projeto, texto_curriculo_completo)